<a href="https://colab.research.google.com/github/wjmanse/FittingroomAnywhere/blob/master/fashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
!git clone https://github.com/matterport/Mask_RCNN

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.90 MiB | 19.71 MiB/s, done.
Resolving deltas: 100% (571/571), done.


In [3]:
!pip install mrcnn

     |████████████████████████████████| 61kB 26.7MB/s 
  Created wheel for mrcnn: filename=mrcnn-0.2-cp36-none-any.whl size=54933 sha256=cb83732044b76fa6ef77bd6e933735d063b3890115890646f9d66ca98ef16fe6
  Stored in directory: /root/.cache/pip/wheels/11/ed/28/e550ddc897c04c336b923eae4eb35c9aae993d20ce39d9cc40
Successfully built mrcnn


In [4]:
!pip install imgaug
!pip install Cython
!pip install pycocotools
!pip install kaggle

In [5]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project

import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.path.abspath("/gdrive/My Drive/fashion/mrcnn")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR))  # To find local version
import fashion

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Using TensorFlow backend.


In [0]:
import fashion
config = fashion.fashionConfig()
COCO_DIR =  os.path.join(ROOT_DIR, "fashion")

In [8]:
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                26
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [9]:
train_dataset = fashion.fashionDataset()
train_dataset.load_fashion(COCO_DIR, "train")
train_dataset.prepare()    
print("Images: {}\nClasses: {}".format(len(train_dataset.image_ids), train_dataset.class_names))

val_dataset = fashion.fashionDataset()
val_dataset.load_fashion(COCO_DIR, "val")
val_dataset.prepare()    
print("Images: {}\nClasses: {}".format(len(val_dataset.image_ids), val_dataset.class_names))


loading annotations into memory...
Done (t=1.33s)
creating index...
index created!
Images: 1000
Classes: ['BG', 'short_sleeved_shirt', 'long_sleeved_shirt', 'short_sleeved_outwear', 'long_sleeved_outwear', 'vest', 'sling', 'shorts', 'trousers', 'skirt', 'short_sleeved_dress', 'long_sleeved_dress', 'vest_dress', 'sling_dress']
loading annotations into memory...
Done (t=1.02s)
creating index...
index created!
Images: 100
Classes: ['BG', 'short_sleeved_shirt', 'long_sleeved_shirt', 'short_sleeved_outwear', 'long_sleeved_outwear', 'vest', 'sling', 'shorts', 'trousers', 'skirt', 'short_sleeved_dress', 'long_sleeved_dress', 'vest_dress', 'sling_dress']


In [13]:
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR,
                              config=config)

weights_path = COCO_MODEL_PATH
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True,exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])


Loading weights  /gdrive/My Drive/fashion/mrcnn/mask_rcnn_coco.h5


In [15]:
print("Fine tune all layers")
model.train(train_dataset,val_dataset,learning_rate=config.LEARNING_RATE ,epochs=160,layers='all')

Fine tune all layers

Starting at epoch 0. LR=0.001

Checkpoint Path: /gdrive/My Drive/fashion/mrcnn/logs/fashion20190804T1727/mask_rcnn_fashion_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_bran

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple worker

Epoch 1/160
100/100 [==============================] - 165s 2s/step - loss: 1.2910 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.1780 - mrcnn_class_loss: 0.2157 - mrcnn_bbox_loss: 0.4229 - mrcnn_mask_loss: 0.4717 - val_loss: 3.9976 - val_rpn_class_loss: 0.0367 - val_rpn_bbox_loss: 1.3881 - val_mrcnn_class_loss: 0.3982 - val_mrcnn_bbox_loss: 0.9618 - val_mrcnn_mask_loss: 1.2127
Epoch 2/160
100/100 [==============================] - 100s 1s/step - loss: 1.1864 - rpn_class_loss: 0.0015 - rpn_bbox_loss: 0.1919 - mrcnn_class_loss: 0.1794 - mrcnn_bbox_loss: 0.3387 - mrcnn_mask_loss: 0.4750 - val_loss: 4.2831 - val_rpn_class_loss: 0.0684 - val_rpn_bbox_loss: 1.6961 - val_mrcnn_class_loss: 0.3612 - val_mrcnn_bbox_loss: 0.8556 - val_mrcnn_mask_loss: 1.3018
Epoch 3/160
100/100 [==============================] - 102s 1s/step - loss: 1.6467 - rpn_class_loss: 0.0087 - rpn_bbox_loss: 0.6461 - mrcnn_class_loss: 0.1745 - mrcnn_bbox_loss: 0.3811 - mrcnn_mask_loss: 0.4362 - val_loss: 4.5617 - val_rpn_clas